In [2]:
import numpy as np
np.random.seed(13)

import random

import keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Flatten
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams
from tqdm import tqdm

import gensim

Using TensorFlow backend.


In [3]:
def load_data(path):
    print('Load ' + path)
    
    data = np.load(path)

    labels = list()
    titles = dict()
    embeddings = list()
    context = list()
    for data_dict in data:

            doc_id = data_dict['doc_index']
            titles[doc_id] = data_dict['doc_title']
            ctx = data_dict['doc_window']

            e = data_dict['pivot_embeddings']

            for emb in e:
                labels.append(doc_id)
                embeddings.append(emb)
                
                neg_ctx = negative_samples(data, ctx, 1)
                context.append(neg_ctx)

    return labels, embeddings, context, titles


def negative_samples(data, doc_context, num):
    negative_samples = []
    
    while len(negative_samples) < num:
        sample = random.choice(data)
    
        if sample['doc_index'] not in doc_context:
            negative_samples.append(sample['pivot_embeddings'][0])
    
    return negative_samples
    

def shuffle_data(x, y, c):
    shuffled = list(zip(x, y, c))
    random.shuffle(shuffled)

    x, y, c = zip(*shuffled)

    return x, y, c

In [17]:
labels, embeddings, context, titles = load_data('/Users/julianbrendl/Projects/bachelor-thesis/word2doc/data/word2doc/1-wpp.npy')

Load /Users/julianbrendl/Projects/bachelor-thesis/word2doc/data/word2doc/1-wpp.npy


In [18]:
embeddings, labels, context = shuffle_data(embeddings, labels, context)

In [35]:
max(labels)

5316192

In [6]:
model = Sequential()

# Hidden layer 1
model.add(Dense(300, input_dim=4096))
model.add(Activation('sigmoid'))

# Output layer
model.add(Dense(5316192))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [174]:
for elem in list(zip(embeddings, context, labels)):
    x = np.array(elem[0], dtype='float32')
    c = np.array(elem[1], dtype='float32')
    y = np.array(elem[2], dtype='float32')
    print(len(x))
    print(y)
    
    break

4096
4034639.0


In [21]:
embeddings = np.asarray(embeddings)
labels = np.array(labels)

In [ ]:
# Train model
model.fit(embeddings, labels,
          epochs=1,
          batch_size=8,
          verbose=2)

print(loss)

Epoch 1/1
